# Predicción de múltiples series de tiempo aplicado en la proyección de matrícula escolar.

#### Importante:
La presente libreta es una herramienta de documentación para la elaboración de la tesis, no pretende ser la implementación final del sistema de proyección de matrícula.
Para ejecutar una celda, dar click en ella y presionar ctrl + enter.

Instalar paquetes necesarios en caso de que no se cuente con ellos. Saltar esta celda si se ejecuta desde Binder.

In [ ]:
# Recargar página luego de ejecutar esta celda por primera vez
!pip install "ipywidgets>=7.2"
!pip install sklearn
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install bqplot
!pip install datapane
!pip install pyFTS
!pip install dill
!pip install matplotlib
!pip install altair
!jupyter nbextension enable --py --sys-prefix bqplot

Importar todos los paquetes necesarios para esta libreta.

In [ ]:
import pandas as pd
import numpy as np
import bqplot.pyplot as plt
import tensorflow as tf
from IPython.display import display, HTML, IFrame
import ipywidgets as widgets
import altair as alt
import datapane as dp

from bqplot import (ColorScale, DateColorScale, OrdinalColorScale, 
                    LinearScale, Tooltip)
from ipywidgets import interact, interactive, fixed, interact_manual

# Evaluación de modelos

En las siguientes celdas se presenta una evaluación de los modelos y después se continúa con la descripción de las métricas.

In [ ]:
# Manejo de modulos
import sys, os
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

# Importar clases de normalización
from Entrenamiento.Normalizators import MinMaxNormalizator, DifferencingNormalizator

# Importar las funciones de los modelos
from Metodos.NaiveForecasting import naive_forecasting_predict
from Metodos.LinearRegression import base_linear_regression
from Metodos.FuzzyTimeSeries import hyperopt_fts_predict
from Metodos.AutoARIMA import auto_arima_predict
from Metodos.IndividualANN import individual_ann
from Metodos.ExpertsOpinion import weightless_ep

# Importar interface de modelos
from Evaluation import Model, TestResult

# Crear modelos

model_dict = {
    'Naive forecasting' : Model(naive_forecasting_predict),
    'Simple Linear Regression' : Model(base_linear_regression),
    'Fuzzy Time Series' : Model(hyperopt_fts_predict),
    'Auto ARIMA' : Model(auto_arima_predict),
    'ANN Individual' : Model(individual_ann, args = dict(window_len = 5, normalizators = [MinMaxNormalizator])),
    'Experts Opinion' : Model(weightless_ep, args = dict(experts = [
        (individual_ann, dict(window_len = 5, normalizators = [MinMaxNormalizator])), 
        (hyperopt_fts_predict, dict()), 
        (auto_arima_predict, dict(normalizators = [MinMaxNormalizator]))
    ]))
}

# Crear etiquetas
label_dict = {
    'Todas las primarias' : 'PrimariasCompletas',
    'Primarias públicas' : 'PrimariasPublicas',
    'Primarias privadas' : 'PrimariasPrivadas',
    'Muestra pequeña' : 'Muestra'
}

In [ ]:
# Seleccionar conjunto de datos, método de predicción y número de años a predecir
out = widgets.Output()
evaluacion_dict = None

@interact(
    modelo = widgets.RadioButtons(
        options = [
            'Naive forecasting',
            'Simple Linear Regression',
            'Fuzzy Time Series',
            'Auto ARIMA',
            'ANN Individual',
            'Experts Opinion'
        ],
        description='Modelo',
        disabled=False
    ),
    conjunto = widgets.RadioButtons(
        options = [
            'Todas las primarias', 
            'Primarias públicas', 
            'Primarias privadas',
            'Muestra pequeña'
        ],
        description='Conjunto',
        disabled=False
    ),
    periodo = widgets.RadioButtons(
        options = [
            'Prueba',
            'Validación'
        ],
        description = 'Periodo',
        disabled = False
    ),
)
def preparar_evaluacion(modelo, conjunto, periodo) :
    global evaluacion_dict
    evaluacion_dict = {
        'modelo' : modelo,
        'conjunto' : conjunto,
        'periodo' : periodo,
    }

In [ ]:
# Calcular resultados de la prueba
from MyUtilities import LoadEvaluationPlot
from MyUtilities import LoadHTMLTable

modelo = evaluacion_dict['modelo']
conjunto = evaluacion_dict['conjunto']
periodo = evaluacion_dict['periodo']
model = model_dict[modelo]

prediction_size = 4
model.TEST_SIZE = 4
if periodo == 'Prueba' :
    model.VALIDATION_SIZE = 4
else :
    model.VALIDATION_SIZE = 0

result = model.test_set(label_dict[conjunto], prediction_size, "GruposPrimaria")

LoadEvaluationPlot(result, prediction_size, conjunto, modelo)

out_1 = widgets.Output()
with out_1:
    out_1.clear_output(True)
    plt.show()
    display(HTML(LoadHTMLTable(result.metricas, modelo)))
    
out_1

## Métricas de evaluación

A continuación se presenta una descripción de las métricas seleccionadas.

#### Notas importantes:

Las métricas son calculadas por año de forma independiente, es decir, que se calcula cada métrica para los 5 años predichos en el futuro, tomándose la predicción y el valor real de cada escuela en cada año.

$\hat{Y}:$ vector de predicción del conjunto de escuelas. Sus elementos se expresan como $\hat{y}_i$

$Y$: vector de datos reales (alumnos inscritos) del conjunto de escuelas. Sus elementos se expresan como $y_i$

El error de una predicción se calcula como $E = Y - \hat{Y}$, y puede ser explicado gráficamente en la siguiente figura:

![](DefinicionError.png)

Donde cada punto representa una predicción, la distancia vertical entre cada punto y la línea base de predicción es el error en la predicción. Los errores son negativos si se encuentran por debajo de la línea base de predicción y son positivos si se encuentran por encima de la línea base de predicción. La línea base de predicción representa la predicción ideal sobre la que deberían estar todas las predicciones en un modelo perfecto. El color de los puntos indica a qué año pertenece la predicción, siendo el color verde el primer año predicho.

### Mean absolute error

Métrica que representa el promedio de error de todas las predicciones realizadas. Otorga la ventaja de representar el error en la misma escala en la que se encuentran los datos.

Cálculo:

$$MAE = \frac{1}{n} \sum_{i = 1}^{n} |\hat{y}_i - y_i |$$

donde

$n$ es el número de escuelas

$y_i$ representa el número de alumnos en la escuela $i$ en un año

$\hat{y}_i$ representa la predicción de alumnos de alumnos la escuela $i$ en un año

### Root mean squared error

Similar al MAE esta métrica representa el error en una escala cercana a la escala de los datos pero que funciona mejor cuando existen variaciones en la escala de los datos.

Cálculo:

$$RMSE = \sqrt{ \frac{1}{n} \sum_{i = 1}^{n} (\hat{y}_i - y_i) ^ {2}}$$

donde

$n$ es el número de escuelas

$y_i$ representa el número de alumnos en la escuela $i$ en un año

$\hat{y}_i$ representa la predicción de alumnos de alumnos la escuela $i$ en un año

### Mean absolute percentage error

Métrica estándard que representa el porcentage de error promedio de todas las predicciones realizadas.

Cálculo:

$$MAPE = \frac{1}{n} \sum_{i = 1} ^ n \frac{|\hat{y}_i - y_i|}{y_i}$$


donde

$n$ es el número de escuelas

$y_i$ representa el número de alumnos en la escuela $i$ en un año

$\hat{y}_i$ representa la predicción de alumnos de alumnos la escuela $i$ en un año

### Probabilidad de riesgo

La probabilidad de riesgo es la única métrica no estándard presente en la evaluación de los modelos. Representa la probabilidad de tomar una proyección de matrícula cuyo error supere al promedio de la taza de alumnos por grupo en dicha escuela. Este tipo de proyecciones se consideran de riesgo ya que en caso de incurrir en una de ellas, se estará tomando una decisión que pueda perjudicar los recursos involucrados, por ejemplo, predecir con un error de 40 alumnos en una escuela en la que el promedio la taza de alumnos por grupo es de 20 alumnos puede conducirnos a casos en los que se inviertan recursos en la creación de dos nuevos grupos cuando no es necesario, o el caso contrario, en el que por no asignar los recursos la escuela no cuenta con la infraestructura necesaria para satisfacer la demanda.

Cálculo de la probabilidad de riesgo $PR$:

$$PR = \frac{1}{n} \sum_{i = 1} ^ {n} f(i)$$

$$
f(i)=
\begin{cases}
1 \Leftrightarrow |\hat{y}_i - y_i| \geq \frac{1}{T} \sum_{j = 1}^T \frac{y_j}{g_j} \\
0 \Leftrightarrow |\hat{y}_i - y_i| < \frac{1}{T} \sum_{j = 1}^T \frac{y_j}{g_j} \\
\end{cases}
$$ 

donde 

$n$ es el número de escuelas

$y_i$ representa el número de alumnos en la escuela $i$ en un año

$\hat{y}_i$ representa la predicción de alumnos de alumnos la escuela $i$ en un año

$g_j$ representa el número de grupos de la escuela en el año $j$ 

$T$ es el número de años utilizados para el conjunto de entrenamiento



# Proyección de matrícula en una sola escuela

En la siguiente celda se evalúa la proyección de matrícula en una escuela dados su clave de centro de trabajo, modelo de predicción y años a predecir.

#### Importante: solo se aceptan cct de escuelas primarias

In [ ]:
# Introduce el CCT, los años a predecir (5 máximo) y el método de predicción
# Ejemplo de CCT: 32EPR0160B
out = widgets.Output()
reporte_dict = None

@interact(cct = 'Introduce cct', 
          anios = widgets.IntSlider(min=1, max=5, step=1, value=3, description = 'Años'), 
          modelo = widgets.RadioButtons(
              options = [
                  'Naive forecasting',
                  'Simple Linear Regression',
                  'Fuzzy Time Series',
                  'Auto ARIMA',
                  'ANN Individual',
                  'Experts Opinion'
              ],
              description='Modelo',
              disabled=False
))
def prediccion(cct, anios, modelo) :
    global model_dict, reporte_dict
    model = model_dict[modelo]
    
    dataset = pd.read_csv(os.path.join(
        os.path.abspath(os.pardir), 
        'Datasets/', 
        'PrimariasCompletas.csv'
    ))
    unique_index = pd.Index(list(dataset['cct']))
    if cct in unique_index :
        index = unique_index.get_loc(cct)
        print('Se encontró el cct')
    else :
        print('No se encontró el cct')
        return
    
    # Quitar el cct
    row = np.array(dataset.loc[index][1:])
    
    # Separamos los últimos 5 años
    X = row[: -model.TEST_SIZE]

    if anios == model.TEST_SIZE :
        Y = row[-model.TEST_SIZE :]
    else :
        Y = row[-model.TEST_SIZE : -(model.TEST_SIZE - anios)]
    prediccion = model.predict(X, anios)
    
    list_X = list(X)
    list_Y = list(Y)
    list_predict = list(prediccion)
    
    # Preparar reporte
    reporte_dict = {
        'title' : 'Predicción de matrícula escolar en la escuela %s utilizando %s' % (cct, modelo),
        'X' : X,
        'Y' : Y,
        'prediccion' : prediccion,
        'anios' : anios
    }
    
    # Mostrar gráfica
    with out:
        out.clear_output(True)
        
        axes_options = {'x': {'label': 'Año'},'y': {'label': 'Alumnos'}}
        
        fig = plt.figure(title = 'Predicción de matrícula escolar en la escuela %s utilizando %s' % (cct, modelo), legend_location='top-left')
        plt.plot(x = [1998 + i for i in range(len(X))], y = list_X, colors = ['red'], axes_options=axes_options, labels = ['Datos de entrenamiento'], display_legend = True, marker = 'circle')
        plt.plot(x = [1997 + len(X) + i  for i in range(len(Y) + 1)], y = (list_X + list_Y)[-(anios + 1):], labels = ['Datos de prueba'], display_legend = True, marker = 'circle')
        plt.plot(x = [1997 + len(X) + i  for i in range(len(Y) + 1)], y = (list_X + list_predict)[-(anios + 1):], colors = ['green'], labels = ['Datos predichos'], display_legend = True, marker = 'circle')
        
        plt.show()
out

## Compartir reporte en Datapane

En la siguientes celdas se pueden crear reportes de la proyección hecha anteriormente y compartirlas públicamente.

In [ ]:
# Escribir aquí el token de acceso de Datapane
# Puedes encontrarlo en: https://datapane.com/home/

!datapane login --server=https://datapane.com/ --token=TOKEN

In [ ]:
from MyUtilities import LoadReport

# Crear reporte
reporte = LoadReport(**reporte_dict)

# Ver reporte en esta celda de Jupyter
reporte.preview()

# Alternativa: guardar reporte en disco y mostrar IFrame en Jupyter
#reporte.save(path='Reporte.html', open = True)
#display(IFrame(src = 'Reporte.html', width = '100%', height = '540px'))

# Publicar
reporte.publish(name='Reporte', open=True, visibility='PUBLIC')